<a href="https://colab.research.google.com/github/sriyathota/mood_guard/blob/main/mood_guard/MoodGuard_HackGT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install tensorflow
!pip install scikit-learn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
##IMPORTS AND LOADING DATA IN
import pandas as pd
from google.colab import drive
import os
import transformers as tf
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import numpy as np
import re
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertForSequenceClassification


drive.mount('/content/drive')
file_path = '/content/drive/My Drive/new_sentiment_data.csv'

df = pd.read_csv(file_path)
df = df.sample(frac=1).reset_index(drop=True)
df = df[0:10000]

print(df)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
      Unnamed: 0                                          statement  \
0          14356  hey, i want to die but I hate physical pain. l...   
1          18668  I am constantly in a state of anxiety. I canno...   
2           2219                  nct who made my nervous comeback.   
3          46039  Help: Explaining to my family Hello! I was rec...   
4           8350  I am just done. I have accepted that I do not ...   
...          ...                                                ...   
9995        4053                                        QUE JYP QUE   
9996       39565  did i give myself erp year ago i think i did w...   
9997       28892  *Major trigger warning* I find comfort in conf...   
9998       52526  Has anyone taken prozak? Has anyone taken proz...   
9999       39445  hello i m having some trouble understanding my...   

          status  

In [ ]:
#DATA PREPROCESSING:

#remove any rows that have missing values
df.dropna(inplace=True)


# Extract text and labels
sentences = df['statement'].values
labels = df['status'].values

#split data into training and testing sets
from sklearn.model_selection import train_test_split


# Function to clean text - make lowercase, remove punctuation
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    return text

# Clean sentences
df['cleaned_statements'] = df['statement'].apply(clean_text)

# Split the dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

"""
train_texts, test_texts, train_labels, test_labels = train_test_split(
    sentences, labels, test_size=0.2
)
"""
#import TfidfVectorizer:
from sklearn.feature_extraction.text import TfidfVectorizer

#vectorization for first column (sentences)
ct = ColumnTransformer(
    transformers=[
        ('tfidf', TfidfVectorizer(), 'cleaned_statements'),  # Use TF-IDF on the 'statement' column
    ],
    remainder='drop'  # Drop any columns not specified in the transformers
)
transformed_data = ct.fit_transform(df)
print(transformed_data)

#encoding second column(status)
label_encoder = LabelEncoder()
df['status_encoded'] = label_encoder.fit_transform(df['status'])
print(df['status_encoded'])
print(df.shape[0])

  (0, 11221)	0.11511417920459249
  (0, 26774)	0.053315452788430744
  (0, 25021)	0.18872258733549113
  (0, 6456)	0.08247287377306887
  (0, 3407)	0.04147184129684775
  (0, 10875)	0.08099683386135056
  (0, 18274)	0.2322800203901963
  (0, 17640)	0.2591506254717433
  (0, 14085)	0.16117804753038195
  (0, 23469)	0.135217600160362
  (0, 7310)	0.10191479350879139
  (0, 25236)	0.135217600160362
  (0, 15721)	0.09801609826469272
  (0, 991)	0.13662242283178544
  (0, 10329)	0.1577529000883943
  (0, 21313)	0.08776003955662313
  (0, 6906)	0.08787081010415079
  (0, 27839)	0.0548750287245449
  (0, 10592)	0.10956671077182843
  (0, 13554)	0.05374161619832288
  (0, 16993)	0.07640427490206061
  (0, 1224)	0.13434632216873463
  (0, 26885)	0.21575356947543733
  (0, 3538)	0.05642903950613428
  (0, 9713)	0.13348598335448306
  :	:
  (9926, 11117)	0.09311698629968618
  (9926, 26394)	0.2508157152429451
  (9926, 11082)	0.07976546113022485
  (9926, 9721)	0.10986157930043092
  (9926, 9064)	0.061509156206008005
  (9926

In [ ]:
#TOKENIZATION

df = pd.read_csv(file_path)

# Extract the "statement" column from the DataFrame *****PUT IN CLEANED STATEMENS WHEN COMBINING CODE
train_statements = train_df['statement'].astype(str).tolist()
test_statements = test_df['statement'].astype(str).tolist()

#BERT TOKENIZER INSTALLATION:
from transformers import BertTokenizer, BertModel

# LOAD PRE-TRAINED MODEL:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#TOKENIZING TEXT DATA:
train_inputs = tokenizer(train_statements,
                   padding=True,           # Pads sequences to the same length
                   truncation=True,         # Truncates long sequences
                   max_length=512,          # Max length for BERT
                   return_tensors='pt')     # Return PyTorch tensors
print(train_inputs)
test_inputs = tokenizer(test_statements,
                   padding=True,           # Pads sequences to the same length
                   truncation=True,         # Truncates long sequences
                   max_length=512,          # Max length for BERT
                   return_tensors='pt')     # Return PyTorch tensors


# Extract tokenized inputs
train_input_ids = train_inputs['input_ids']
train_attention_mask = train_inputs['attention_mask']

test_input_ids = test_inputs['input_ids']
test_attention_mask = test_inputs['attention_mask']

# Extracting unique labels from csv column "status"
train_labels = train_df['status'].tolist()  # This will be a Numpy Array
test_labels = test_df['status'].tolist()  # This will be a Numpy Array

# Create a LabelEncoder instance
label_encoder = LabelEncoder()

# Fit the encoder to the labels and transform them into numerical format
encoded_train_labels = label_encoder.fit_transform(train_labels)  # This will convert each label to a unique integer
encoded_test_labels = label_encoder.fit_transform(test_labels)



#creates dictionary to hold connection between string label & integer assigned by LabelEncoder()
label_mapping = {label: index for index, label in enumerate(label_encoder.classes_)}
print("Label mapping:", label_mapping)

# Convert input_ids, attention_mask, and labels to torch tensors
#TORCH TENSORS --> data structue (arrays/matrices rather than python lists) which can be fed into model
train_input_ids_tensor = torch.tensor(train_input_ids, dtype=torch.long)
train_attention_mask_tensor = torch.tensor(train_attention_mask, dtype=torch.long)
train_labels_tensor = torch.tensor(encoded_train_labels, dtype=torch.long)

test_input_ids_tensor = torch.tensor(test_input_ids, dtype=torch.long)
test_attention_mask_tensor = torch.tensor(test_attention_mask, dtype=torch.long)
test_labels_tensor = torch.tensor(encoded_test_labels, dtype=torch.long)

# Create a TensorDataset
train_dataset = TensorDataset(train_input_ids_tensor, train_attention_mask_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_input_ids_tensor, test_attention_mask_tensor, test_labels_tensor)

# Create a DataLoader to load the data in batches
batch_size = 16  # You can adjust this based on memory capacity
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Load pre-trained BERT with a classification layer for your task (adjust num_labels)
num_labels = len(label_mapping)  # Number of unique labels in your dataset

#BERT(pre-trained model): learns the patterns of statements to sentiments during training to later predict the sentiment of new statements
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)












/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'input_ids': tensor([[  101, 14736,  5003,  ...,     0,     0,     0],
        [  101, 23760,  3366,  ...,     0,     0,     0],
        [  101,  1996,  2069,  ...,     0,     0,     0],
        ...,
        [  101,  1045,  2202,  ...,     0,     0,     0],
        [  101,  3782,  9336,  ...,     0,     0,     0],
        [  101,  4931,  2417,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
Label mapping: {'Anxiety': 0, 'Bipolar': 1, 'Depression': 2, 'Normal': 3, 'Personality disorder': 4, 'Stress': 5, 'Suicidal': 6}


<ipython-input-22-463a2c8d7e12>:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_input_ids_tensor = torch.tensor(train_input_ids, dtype=torch.long)
<ipython-input-22-463a2c8d7e12>:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_attention_mask_tensor = torch.tensor(train_attention_mask, dtype=torch.long)
<ipython-input-22-463a2c8d7e12>:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_input_ids_tensor = torch.tensor(test_input_ids, dtype=torch.long)
<ipython-input-22-463a2c8d7e12>:61: UserWarning: To copy construct from

In [ ]:
#TRAINING BERT

from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from transformers import get_linear_schedule_with_warmup

# defining the optimizer and the learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)

# the t otal number of training steps = number of batches * number of epochs
epochs = 3
total_steps = len(train_dataloader) * epochs

# learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)

#CrossEntroyLoss measures model's predictions vs true labels (accuracy)
loss_fn = CrossEntropyLoss()


#checks if GPU is available, otherwise uses a CPU *moves model to whichever one is available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

for epoch in range(epochs):
    model.train()  # Set model to training mode

    #tracks model's performance for each batch
    total_loss = 0

    for batch in train_dataloader:
        # unpack the batch
        input_ids_batch, attention_mask_batch, labels_batch = [b.to(device) for b in batch]

        # clear previous calculated gradients
        model.zero_grad()

        #forward pass (with automatic gradient calculation)
        #for a given batch, unpacks it into 3 part and moves it to either GPU/CPU
        outputs = model(input_ids=input_ids_batch, attention_mask=attention_mask_batch, labels=labels_batch)
        #convert output items from long to float
        target = [0,1,2,3,4,5]
        loss = loss_fn(outputs.logits, labels_batch)

        # fackward pass
        # compares the model's predicted label classification to the real given label & calculates loss (accuracy)
        # computes gradient of loss (change of loss with respect to change in paramter *input data)
        loss.backward()

        # clips the gradients to prevent exploding gradients
        # caps/limits graident scale to max value of 1.0
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # update weights
        optimizer.step()

        # adjusts learning rate
        scheduler.step()

        # Track loss
        #adds current batch loss to the total for each epoch
        total_loss += loss.item()
        #print("batch")

   #prints avg loss at end of each epoch
   #provides insight into how well the model is learning over time
    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}, Loss: {avg_loss}')

    #TESTING
    # sets model to evaluation mode
    model.eval()

    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad(): #disable gradient calculation during evaluation
        for batch in test_dataloader:
            input_ids_batch, attention_mask_batch, labels_batch = [b.to(device) for b in batch]

            #forward pass
            outputs = model(input_ids=input_ids_batch, attention_mask=attention_mask_batch)

            # get the predicted labels
            predicted_labels = torch.argmax(outputs.logits, dim=-1)

            # calculate accuracy
            correct_predictions += (predicted_labels == labels_batch).sum().item()
            total_predictions += labels_batch.size(0)

    accuracy = correct_predictions / total_predictions
    print(f'Accuracy: {accuracy:.2f}')

    # saving the model to a directory!!
    model_save_path = '/content/drive/My Drive/mood_guard'
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)
    print(f'Model saved to: {model_save_path}')


Epoch 1, Loss: 0.9580651595889325
Accuracy: 0.75
Model saved to: /content/drive/My Drive/mood_guard
Epoch 2, Loss: 0.5391891321025221
Accuracy: 0.79
Model saved to: /content/drive/My Drive/mood_guard
Epoch 3, Loss: 0.41204572603645095
Accuracy: 0.80
Model saved to: /content/drive/My Drive/mood_guard


In [ ]:
#IMPLEMENTATION / TESTING MODEL WITH USER INPUT

from transformers import BertForSequenceClassification, BertTokenizer

# loading the model and tokenizer from the specified path
model = BertForSequenceClassification.from_pretrained('/content/drive/My Drive/mood_guard')
tokenizer = BertTokenizer.from_pretrained('/content/drive/My Drive/mood_guard')

sample_text = "I'm very depressed"

# preprocessing the input text
inputs = tokenizer(sample_text,
                   padding=True,            # Pads to the maximum length in batch
                   truncation=True,         # Truncates if too long
                   max_length=512,          # Maximum length for BERT
                   return_tensors='pt')     # Return PyTorch tensors

# make predictions
model.eval()  # this sets the model to evaluation mode
with torch.no_grad():  # disables gradient calculation
    outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
    logits = outputs.logits  # get the logits from the model's output

# converts logits to probabilities stats using softmax
probs = torch.softmax(logits, dim=1)  # Apply softmax to get probabilities

# getting the predicted class
# gets the index of the max probability
predicted_class = torch.argmax(probs, dim=1).item()

# mapping the predicted class index back to the original label
label_mapping = {0: 'Anxiety', 1: 'Bipolar', 2: 'Depression', 3: 'Normal', 4: 'Personality Disorder', 5: 'Stress', 6: 'Suicidal'}  # Example mapping


predicted_label = label_mapping[predicted_class]

# output the result
print(f'The predicted category for the input text is: {predicted_label}')



The predicted category for the input text is: Anxiety


In [ ]:
!pip install gradio

In [ ]:
#GUI
import gradio as gr

from __future__ import annotations
from typing import Iterable
import gradio as gr
from gradio.themes.base import Base
from gradio.themes.utils import colors, fonts, sizes
import time

my_theme = gr.themes.Monochrome(
    primary_hue="cyan",
    secondary_hue="teal",
    neutral_hue="teal",
).set(
    body_background_fill='*color_accent_soft',
    background_fill_primary='*primary_50',
    border_color_primary='*neutral_700'
)

#puts pop up on the screen
def info_fn(user_input):
    response = call_model(user_input)
    #fetch the specific message for the predicted category
    message = category_messages.get(response, "Unable to determine a category. Please try again.")
    return gr.Info(message)

# dictionary: mapping each category to a specific message
category_messages = {
    "Anxiety": "It looks like you're feeling anxious. Consider practicing relaxation techniques. You can also call or text a NAMI representative for free at 800-950-6264. <a href='https://open.spotify.com/playlist/3l6b0zuXjgyPxLK6PIAqED?si=0ywZftzXQJmGAJyNGwaxUA'>Check out this Spotify playlist for more support.<a/>",
    "Bipolar": "Managing bipolar disorder can be challenging. Keeping a regular routine can help. You can contact the Depression and Bipolar Support Alliance at 312-642-0049. <a href='https://open.spotify.com/playlist/1UXZQs7qhBMnFYO00QxQtd?si=aJP_VostSV-z3kOYRxrt9Q&pi=u-17BWOZ9ZSOuI&nd=1&dlsi=5a70ab4d0f9d4538'>Check out this Spotify playlist for more support.<a/>",
    "Depression": "It seems you're feeling depressed. Talking to a counselor or therapist might help. You can contact the Depression and Bipolar Support Alliance at 312-642-0049. <a href='https://open.spotify.com/playlist/7INcD4lmarWTQiDVodjVt4?si=2cS5G230REqV3kD2MA28nQ&pi=u-2PmAr4bFQn2X&nd=1&dlsi=b7742602f12d4430'>Check out this Spotify playlist for more support.<a/>",
    "Normal": "Make sure to keep taking care of your mental health! If you want to help others in need, you can volunteer at Lifeline, CHMA, or CASA! <a href='https://open.spotify.com/playlist/6ghnEF33uY7YCgnum8NWAR?si=QnPO4vo3SbGI0BRlRgfiRA&pi=u-VxAETTCLSU60&nd=1&dlsi=5f5516b9657a4385'>Check out this Spotify playlist for more support.<a/>",
    "Stress": "You might be feeling stressed. Try mindfulness or take short breaks to relax. You can also contact the National Mental Health Hotline for free at 866-903-3787. <a href='https://open.spotify.com/playlist/6Ttw2Vt8F5zRqYSb4qcmTo?si=FUjz5r_QTrSPsmCcT1jNhw&pi=u-1pjEp6QjS4O-&nd=1&dlsi=c674a614103b46b7'>Check out this Spotify playlist for more support.<a/>",
    "Suicidal": "Please seek immediate help from a trusted person or a helpline. You can contact the Suicide Lifeline at 1-800-273-8255. <a href='https://open.spotify.com/playlist/7INcD4lmarWTQiDVodjVt4?si=2cS5G230REqV3kD2MA28nQ&pi=u-2PmAr4bFQn2X&nd=1&dlsi=8bb79cc179ef46af'>Check out this Spotify playlist for more support.<a/>"
}

#passes the user input to the model so it can classify what the user is feeling
def call_model(user_input):
    from transformers import BertForSequenceClassification, BertTokenizer

    #loads the model and tokenizer from the specified path
    model = BertForSequenceClassification.from_pretrained('/content/drive/My Drive/mood_guard')
    tokenizer = BertTokenizer.from_pretrained('/content/drive/My Drive/mood_guard')
    #preprocess the input text
    inputs = tokenizer(user_input,
                      padding=True,            # Pads to the maximum length in batch
                      truncation=True,         # Truncates if too long
                      max_length=512,          # Maximum length for BERT
                      return_tensors='pt')     # Return PyTorch tensors

    #make predictions
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient calculation
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
        logits = outputs.logits  # Get the logits from the model's output

    #convert logits to probabilities using softmax
    probs = torch.softmax(logits, dim=1)  # Apply softmax to get probabilities

    #get the predicted class
    predicted_class = torch.argmax(probs, dim=1).item()  # Get the index of the max probability

    #map the predicted class index back to the original label
    label_mapping = {0: 'Anxiety', 1: 'Bipolar', 2: 'Depression', 3: 'Normal', 4: 'Personality Disorder', 5: 'Stress', 6: 'Suicidal'}  # Example mapping


    predicted_label = label_mapping[predicted_class]

    #output the result
    return predicted_label


with gr.Blocks(theme=my_theme) as demo:
    gr.Markdown("### MoodGuard")

    #textbox for user input
    user_input = gr.Textbox(label="Enter a message")

    #submit button
    submit_button = gr.Button("Submit")

    #sets up the button click event
    submit_button.click(info_fn, inputs=user_input)


#iface = gr.Interface(fn=greet, inputs="text", outputs="text")
#iface.launch()
if __name__ == "__main__":
  demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://05c3e6fc19daffc4c3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
